In [22]:
import pandas as pd
dfcount = pd.read_csv("0667ba91-d9e6-4f8e-b246-fc77ec36ba77.csv")
dfcount

,content_languages,count
0,eng,982966944
1,NaN,585787318
2,rus,107580325
3,zho,90350792
4,deu,86497311
...,...,...
218506,"eng,mal,mlt",1
218507,"ind,san,aze",1
218508,"tat,deu,msa",1
218509,"srp,mlg,sqi",1


,id,n_commoncrawl
0,eng,1647268067
1,rus,175309697
2,zho,158700611
3,deu,161337878
4,spa,138488278
...,...,...
156,sag,11777
157,sux,2087
158,got,1289
159,kas,637


,ISO 639-2 Code,ISO 639-1 Code,English name of Language,French name of Language,German name of Language\n
0,aar,aa,Afar,afar,Danakil-Sprache\n
1,abk,ab,Abkhazian,abkhaze,Abchasisch\n
2,ace,,Achinese,aceh,Aceh-Sprache\n
3,ach,,Acoli,acoli,Acholi-Sprache\n
4,ada,,Adangme,adangme,Adangme-Sprache\n
...,...,...,...,...,...
502,znd,,Zande languages,"zandé, langues",Zande-Sprachen\n
503,zul,zu,Zulu,zoulou,Zulu-Sprache\n
504,zun,,Zuni,zuni,Zuñi-Sprache\n
505,zxx,,No linguistic content; Not applicable,pas de contenu linguistique; non applicable,Kein linguistischer Inhalt\n


In [34]:
from collections import defaultdict

# read number of pages per language in commoncrawl
counts = {}
for ix, row in dfcount.query("content_languages.notna()").iterrows():
    lang = row["content_languages"]
    lang = lang.split(",")
    for l in lang:
        if l not in counts.keys():
            counts[l] = 0
        counts[l] += row["count"]

counts = {"id": list(counts.keys()), "n_commoncrawl": list(counts.values())}
counts = pd.DataFrame(counts)

# read numbers of speakers from https://www.languagecourse.net/languages-worldwide
df = [line.split("\t") for line in open("isocodes.tsv")][1:]
df = list(filter(lambda x:len(x)==len(df[0]), df))
df = pd.DataFrame(df[1:], columns=df[0])

df = [line.split("\t") for line in open("speakers.tsv")]
df = list(filter(lambda x:len(x) == len(df[0]), df))
columns = [x.strip() for x in df[0]]
df = pd.DataFrame(df[1:], columns=columns)
df = df[["Name", "ISO Alpha-3/5", "Speakers worldwide"]]
columns = {
    'Name': "name", 
    'ISO Alpha-3/5': "id",
    'Speakers worldwide': "n_speakers"
}
df = df.rename(columns=columns).query("n_speakers.notna() and id.notna()")


In [36]:

# merge
num_speakers = {}
lang_names = {}
for ix, row in df.iterrows():
    num_speakers[row["id"]] = row["n_speakers"]
    lang_names[row["id"]] = row["name"]

n_speakers = []
l_names = []
for id in counts.id:
    if id in num_speakers.keys():
        n_speakers.append(num_speakers[id])
    else:
        n_speakers.append(pd.NA)
    if id in lang_names.keys():
        l_names.append(lang_names[id])
    else:
        l_names.append(pd.NA)

counts["name"] = l_names
counts["n_speakers"] = n_speakers

counts = counts.query("n_speakers.notna() and n_speakers != 'No data'")
counts["n_speakers"] = counts.n_speakers.astype(int)

counts = counts[['name', 'id', 'n_commoncrawl', 'n_speakers']]

n_commoncrawl_norm = counts.n_commoncrawl / counts.n_commoncrawl.max()
n_speakers_norm = counts.n_speakers / counts.n_speakers.max()
counts["rating"] = n_commoncrawl_norm * n_speakers_norm
counts = counts.sort_values("rating", ascending=False)

counts.head(50)

,name,id,n_commoncrawl,n_speakers,rating
55,Urdu,urd,1178093,162642730,0.310021
50,Marathi,mar,920966,73000000,0.108779
94,Javanese,jav,450862,100000000,0.072949
79,Panjabi,pan,371661,100000000,0.060135
49,Malayalam,mal,922122,38000000,0.056696
69,Uzbek,uzb,1186466,27000000,0.051832
42,Nepali,nep,1636019,16000000,0.042353
40,Tatar,tat,2802261,6500000,0.029471
97,Pashto,pus,276429,60000000,0.026836
64,Kazakh,kaz,1223461,11000000,0.021775


,id,n_commoncrawl,name,n_speakers
131,tsn,77467,Tswana,No data
117,tuk,237750,Turkmen,No data
86,gla,270758,Scottish Gaelic,No data
105,mlg,275700,Malagasy,No data
59,nno,4255222,Norwegian Nynorsk,No data
...,...,...,...,...
137,aka,53497,Akan,11000000
135,uig,67384,Uyghur,10400000
94,jav,450862,Javanese,100000000
79,pan,371661,Panjabi,100000000
